<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/multi_stock_ppo_live_inference_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y
!apt-get clean

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cuda-toolkit-12-4-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-7' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-8' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-5-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-2' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-3' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-4' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-5' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-6' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-8' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-9' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-6-config

In [2]:
#Protocol Buffer Fix (for TensorFlow)
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


In [3]:
#Update Colab Environment and System Libraries
!apt-get update -y && apt-get upgrade -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,853 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,773 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [4]:
#Install Correct Version of CUDA for Colab GPU
!apt-get update -qq && apt-get install -y \
    libcusolver11 libcusparse11 libcurand10 libcufft10 libnppig10 libnppc10 libnppial10 \
    cuda-toolkit-12-4

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libnppig10
E: Unable to locate package libnppc10
E: Unable to locate package libnppial10


In [5]:
#Set Correct CUDA Paths
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'
os.environ['PATH'] += ':/usr/local/cuda-12.4/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.4/lib64'


In [6]:
#Install RAPIDS and NVIDIA Dependencies
!pip install --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==25.2.0 cudf-cu12==25.2.0 cupy-cuda12x dask-cuda==25.2.0 dask-cudf-cu12==25.2.0


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.0/405.0 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.2/851.2 kB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.7/557.7 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.

In [1]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0

#Install Stable Baselines3 and Trading Libraries
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib

#Install Miscellaneous Libraries
!pip install matplotlib scikit-learn pandas numba==0.61.0

#Install PyTorch with GPU Support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124


In [2]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0


import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow GPU memory growth enabled")
    except RuntimeError as e:
        print(f"TensorFlow GPU memory config failed: {e}")


In [3]:
# Install all required packages
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance --quiet
!pip install stable-baselines3[extra] --quiet


In [12]:
!rm -rf /content/drive

rm: cannot remove '/content/drive/MyDrive': Operation canceled
rm: cannot remove '/content/drive/.shortcut-targets-by-id': Operation canceled
rm: cannot remove '/content/drive/.Trash-0': Directory not empty
rm: cannot remove '/content/drive/.Encrypted/MyDrive': Operation canceled
rm: cannot remove '/content/drive/.Encrypted/.shortcut-targets-by-id': Operation canceled


In [11]:
#Step 7: authenticate with hugging face hub (optional)
#This allows for better access and avoids rate limits when downloading public models/datasets

# Authenticate with Hugging Face Hub
#notebook_login()

In [13]:
#  Import Required Libraries
from transformers import pipeline
import gc
import json
import os
import random
import time
from collections import deque
from datetime import datetime, timedelta

import cupy as cp
import cudf
import cuml
import dask
import gymnasium as gym
import gym_anytrading
import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
import torch
import xgboost as xgb
import yfinance as yf
from cuml.ensemble import RandomForestClassifier
from gym_anytrading.envs import StocksEnv
from gymnasium.spaces import Box
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv
import pywt
from google.colab import drive

# === Mount Google Drive ===
drive.mount('/content/drive')

#  Ticker List and CONFIG
ticker_list = [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO',
    'BAC', 'ABBV', 'AVGO', 'PFE', 'COST', 'CSCO', 'TMO', 'ABT', 'ACN', 'WMT',
    'MCD', 'ADBE', 'DHR', 'CRM', 'NKE', 'INTC', 'QCOM', 'NEE', 'AMD', 'TXN',
    'AMGN', 'UPS', 'LIN', 'PM', 'UNP', 'BMY', 'LOW', 'RTX', 'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL'
]

strategy_name = "ppo_walkforward_v1"
CONFIG = {
    'symbols': ticker_list,
    'period': '720d',
    'interval': '1h',
    'target': 'Target',
    'sharpe_threshold': 1.5,
    'return_threshold': 1.25,
    'strategy_name': strategy_name
}
#  Download Function with Date Logic
def download_stock_data(ticker, start_date=None, end_date=None, interval="1h", max_retries=5, window_days=730):
    if start_date is None or end_date is None:
        end_date = datetime.today()
        start_date = end_date - timedelta(days=window_days)

    start_str = start_date.strftime('%Y-%m-%d')
    end_str = end_date.strftime('%Y-%m-%d')

    for attempt in range(1, max_retries + 1):
        try:
            print(f"Attempt {attempt}: Downloading {ticker} from {start_str} to {end_str}...")
            df = yf.download(
                                ticker,
                                start=start_str,
                                end=end_str,
                                interval=interval,
                                progress=False,
                                auto_adjust=False  # ✅ Explicitly set to avoid future warnings
                            )

            if not df.empty:
                df.reset_index(inplace=True)
                df['Symbol'] = ticker
                df['Datetime'] = pd.to_datetime(df['Datetime'] if 'Datetime' in df.columns else df['Date'])
                return df
            raise ValueError("Empty data")
        except Exception as e:
            print(f"Error: {e}. Retrying in {attempt * 5} sec...")
            time.sleep(attempt * 5)

    print(f" Failed to download {ticker}")
    return None

def denoise_wavelet(series, wavelet='db1', level=2):
    coeffs = pywt.wavedec(series, wavelet, mode='smooth')
    coeffs[1:] = [np.zeros_like(c) for c in coeffs[1:]]  # Zero out detail coefficients
    return pywt.waverec(coeffs, wavelet, mode='smooth')[:len(series)]

# === Dynamic Walkforward Feature Engineering Pipeline ===
test_mode = True  # ✅ Set to False to run all tickers
symbols = CONFIG['symbols']
if test_mode:
    symbols = ['AAPL', 'NVDA', 'MSFT']  # Test with a single ticker like TSLA or NVDA

all_dfs = []
end_date = datetime.today()
start_date = end_date - timedelta(days=729)  # Keep within Yahoo's limit

# === FinBERT Sentiment Scoring ===
sentiment_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert")

def score_sentiment(texts):
    results = []
    try:
        outputs = sentiment_pipeline(texts, truncation=True, max_length=256)
        for r in outputs:
            label = r['label'].lower()
            score = r['score'] if label == 'positive' else (-r['score'] if label == 'negative' else 0)

            results.append(score)
    except Exception as e:
        print(f"Sentiment scoring error: {e}")
        results = [0] * len(texts)
    return results

# === Enhanced Feature Engineering ===
def compute_enhanced_features(df):
    df = df.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.loc[:, ~df.columns.duplicated()]

    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['STD_20'] = df['Close'].rolling(20).std()
    df['Upper_Band'] = df['SMA_20'] + 2 * df['STD_20']
    df['Lower_Band'] = df['SMA_20'] - 2 * df['STD_20']
    df['Lowest_Low'] = df['Low'].rolling(14).min()
    df['Highest_High'] = df['High'].rolling(14).max()
    denom = (df['Highest_High'] - df['Lowest_Low']).replace(0, np.nan)
    df['Stoch'] = ((df['Close'] - df['Lowest_Low']) / denom) * 100
    df['ROC'] = df['Close'].pct_change(10)
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    df['CCI'] = ((df['High'] + df['Low'] + df['Close']) / 3 - df['Close'].rolling(20).mean()) / (0.015 * df['Close'].rolling(20).std())
    df['EMA_10'] = df['Close'].ewm(span=10).mean()
    df['EMA_50'] = df['Close'].ewm(span=50).mean()
    df['MACD_Line'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=9).mean()
    df['RSI'] = 100 - (100 / (1 + df['Close'].diff().apply(lambda x: x if x > 0 else 0).rolling(14).mean() /
                              -df['Close'].diff().apply(lambda x: x if x < 0 else 0).rolling(14).mean()))

    tr = pd.concat([
        df['High'] - df['Low'],
        abs(df['High'] - df['Close'].shift()),
        abs(df['Low'] - df['Close'].shift())
    ], axis=1).max(axis=1)
    df['ATR'] = tr.rolling(14).mean()

    df['Volatility'] = df['Close'].pct_change().rolling(20).std()
    df['Return'] = (df['Close'].shift(-10) - df['Close']) / df['Close']
    df['Target'] = np.select([df['Return'] > 0.02, df['Return'] < -0.02], [1, -1], default=0)

    try:
        df['Denoised_Close'] = denoise_wavelet(df['Close'].ffill())
    except Exception as e:
        print(f"Wavelet denoising failed: {e}")
        df['Denoised_Close'] = df['Close']  # fallback

    print(df[['Close', 'Denoised_Close']].head())

    # === Add mock headlines (you can later map real ones by timestamp)
    df['Mock_Headline'] = f"{df['Symbol'].iloc[0]} is expected to perform well in the market."

    # === Score sentiment from FinBERT
    try:
        df['SentimentScore'] = score_sentiment(df['Mock_Headline'].tolist())
    except Exception as e:
        print(f"Sentiment scoring failed for {df['Symbol'].iloc[0]}: {e}")
        df['SentimentScore'] = 0


    df['Delta'] = df['Close'].pct_change(1).fillna(0)
    df['Gamma'] = df['Delta'].diff().fillna(0)

    df.dropna(inplace=True)

    # 🧼 Reorder columns so important ones are at the end for modeling clarity
    cols = [col for col in df.columns if col not in ['Target', 'Return', 'Symbol']] + ['Target', 'Return', 'Symbol']
    df = df[cols]

    return df

for ticker in symbols:
    df_single = download_stock_data(
        ticker=ticker,
        start_date=start_date,
        end_date=end_date,
        interval="1h"
    )

    if df_single is not None and not df_single.empty:
        try:
            df_features = compute_enhanced_features(df_single)
            all_dfs.append(df_features)
        except Exception as e:
            print(f" Feature engineering failed for {ticker}: {e}")
    else:
        print(f" No data for {ticker}")

#  Combine and Save
if all_dfs:
    df = pd.concat(all_dfs, ignore_index=True)
    print(f" Combined dataset created with shape: {df.shape}")
    df.to_csv("multi_stock_feature_engineered_dataset.csv", index=False)
    print(" Saved locally to multi_stock_feature_engineered_dataset.csv")

    drive_path = "/content/drive/MyDrive/trading_data/"
    os.makedirs(drive_path, exist_ok=True)
    df.to_csv(os.path.join(drive_path, "multi_stock_feature_engineered_dataset.csv"), index=False)
    print(f" Also saved to Google Drive at {drive_path}multi_stock_feature_engineered_dataset.csv")
else:
    print(" No usable data found for any ticker.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Device set to use cpu


Attempt 1: Downloading AAPL from 2023-08-06 to 2025-08-04...
Price       Close  Denoised_Close
0      179.035004       192.47772
1      177.770004       192.47772
2      178.365005       192.47772
3      178.335007       192.47772
4      178.449997       192.47772


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Attempt 1: Downloading NVDA from 2023-08-06 to 2025-08-04...
Price      Close  Denoised_Close
0      45.378502       79.874219
1      44.981041       79.874219
2      45.181999       79.874219
3      44.773003       79.874219
4      44.732998       79.874219


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Attempt 1: Downloading MSFT from 2023-08-06 to 2025-08-04...
Price       Close  Denoised_Close
0      330.320007       392.28746
1      328.420013       392.28746
2      329.149994       392.28746
3      329.029999       392.28746
4      329.424988       392.28746


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


 Combined dataset created with shape: (10329, 32)
 Saved locally to multi_stock_feature_engineered_dataset.csv
 Also saved to Google Drive at /content/drive/MyDrive/trading_data/multi_stock_feature_engineered_dataset.csv


In [ ]:
print(df.head())

In [14]:
#  PPO Walkforward with Runtime Timing, Full Model Saving, and Logging
import os
import gc
import time
import json
import torch
import logging
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import VecNormalize

from gym_anytrading.envs import StocksEnv
from gymnasium.spaces import Box

# === Configuration ===
RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/ppo_walkforward_results"
FINAL_MODEL_DIR = os.path.join(RESULTS_DIR, "models")
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)

# === Logging Setup ===
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# === Global Feature Toggles ===
ENABLE_SENTIMENT = True    # Toggle FinBERT sentiment usage
ENABLE_SLO = True          # Toggle Stop-Loss & Take-Profit logic
ENABLE_WAVELET = True      # Toggle Wavelet denoising (if supported in your pipeline)
test_mode = True

# === Load Data ===
if not os.path.exists("multi_stock_feature_engineered_dataset.csv"):
    raise FileNotFoundError("Required feature-engineered dataset not found!")

df = pd.read_csv("multi_stock_feature_engineered_dataset.csv")
df['Datetime'] = pd.to_datetime(df['Datetime'])

# === PPO-Compatible Trading Environment ===
class ContinuousTradingEnv(StocksEnv):
    def __init__(self, df, frame_bound, window_size):
        super().__init__(df=df.reset_index(drop=True), frame_bound=frame_bound, window_size=window_size)
        self.action_space = Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)

    def step(self, action):
        try:
            if action < -0.3:
                discrete_action = 0
            elif action > 0.1:
                discrete_action = 1
            else:
                discrete_action = 2

            obs, reward, terminated, truncated, info = super().step(discrete_action) if len(super().step(discrete_action)) == 5 else (*super().step(discrete_action), False)

            current_price = self.df.loc[self._current_tick, 'Close']
            slippage_pct = 0.002
            fill_price = current_price * (1 + slippage_pct if discrete_action == 1 else (1 - slippage_pct if discrete_action == 0 else 0))
            price_diff = fill_price - self.df.loc[self._current_tick - 1, 'Close']
            reward += price_diff * discrete_action

            action_scalar = float(np.array(action).squeeze())
            confidence_bonus = abs(action_scalar) * 0.01
            reward += confidence_bonus

            if ENABLE_SENTIMENT and 'SentimentScore' in self.df.columns:
                sentiment = self.df.loc[self._current_tick, 'SentimentScore']
                reward += sentiment * 0.01

            return obs, np.clip(reward, -1.0, 1.0), terminated, truncated, info
        except Exception as e:
            logging.error(f"Step error: {e}")
            return self.reset(), 0, True, True, {}

# === Plot Function ===
def plot_performance(ticker, portfolio, hold_value):
    plt.figure(figsize=(10, 4))
    plt.plot(portfolio, label="PPO Portfolio")
    plt.axhline(y=hold_value, color="r", linestyle="--", label="Buy & Hold")
    plt.title(f"{ticker} - PPO vs Buy & Hold")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, f"{ticker}_performance.png"))
    plt.close()

# === Walkforward Window Generator ===
def get_walk_forward_windows(df, window_size=3000, step_size=500, min_len=1000):
    return [
        (start, start + window_size)
        for start in range(0, len(df) - min_len, step_size)
        if start + window_size < len(df)
    ]

# === PPO Walkforward Function ===
def walkforward_ppo(df, ticker, window_size=3000, step_size=500, timesteps=1_500_000, learning_rate=2e-5):
    if len(df) < window_size:
        logging.warning(f"Skipping {ticker}: only {len(df)} rows (min required: {window_size})")
        return []

    results = []
    windows = get_walk_forward_windows(df, window_size, step_size)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    for w_idx, (start, end) in enumerate(windows):
        window_start_time = time.time()
        gc.collect()
        df_window = df.iloc[start:end].reset_index(drop=True)
        if len(df_window) <= 52 or len(df_window) % 2 != 0:
            df_window = df_window.iloc[:-1]

        logging.info(f"{ticker} | Window {w_idx+1}/{len(windows)} | Rows: {len(df_window)}")
        frame_bound = (50, len(df_window) - 3)

        env = DummyVecEnv([lambda: ContinuousTradingEnv(df=df_window, frame_bound=frame_bound, window_size=10)])
        env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)
        model = PPO(
            "MlpPolicy",
            env,
            verbose=0,
            device=device,
            learning_rate=learning_rate,  # ← e.g. 5e-5
            n_steps=2048,
            batch_size=64,
            n_epochs=10,
            gamma=0.99,
            gae_lambda=0.95,
            clip_range=0.2,
            ent_coef=0.01,
            policy_kwargs=dict(net_arch=[64, 64])
        )

        model.learn(total_timesteps=timesteps)

        # === Save PPO model, VecNormalize state, and feature list
        prefix = f"ppo_{ticker}_window{w_idx+1}"

        # 1. Save model
        model.save(os.path.join(FINAL_MODEL_DIR, f"{prefix}.zip"))

        # 2. Save VecNormalize stats
        if isinstance(env, VecNormalize):
            env.save(os.path.join(FINAL_MODEL_DIR, f"{prefix}_vecnorm.pkl"))

        # 3. Save features used
        json.dump(
            {"features": df_window.columns.tolist()},
            open(os.path.join(FINAL_MODEL_DIR, f"{prefix}_features.json"), "w")
        )

        # === Inference and Portfolio Calculation
        TRADE_COOLDOWN_STEPS = int(0.3 * len(df_window) / 100)
        obs = env.reset()
        action_vals = []
        position, balance, portfolio, trade_log = 0, 100000, [], []
        total_reward = 0
        trade_returns = []
        last_trade_step = -10  # arbitrary negative so we don't penalize the first trade
        whipsaw_penalty = 0.0
        entry_price = None  # Track entry price globally
        risk_flags = []     # Track triggered risk events per step
        cooldown_skips = []
        for i in range(len(df_window)):
            action, _ = model.predict(obs, deterministic=True)
            step_result = env.step(action)
            obs, reward, terminated, truncated, info = step_result if len(step_result) == 5 else (*step_result, False)

            action_val = float(np.array(action).squeeze())
            action_vals.append(action_val)
            price = df_window['Close'].iloc[i]  # ✅ Move this up

            if i % 100 == 0:
                print(f"[Step {i}] Action: {action_val:.4f} | Reward: {reward.item():.4f} | Price: {price:.2f}")

            logging.debug(f"Step {i} | Action: {action_val:.4f} | Reward: {reward.item():.4f} | Position: {position} | Price: {price}")

            # === Risk Management Module ===
            STOP_LOSS_PCT = 0.04
            TAKE_PROFIT_PCT = 0.12
            MAX_EXPOSURE = 1.0  # Placeholder for future exposure control
            risk_triggered = None  # Track risk condition this step
            if position > 0 and entry_price is not None:
                price_change = (price - entry_price) / entry_price

                if price_change < -STOP_LOSS_PCT:
                    balance = position * price
                    position = 0
                    trade_log.append("STOP-LOSS")
                    reward -= 0.2
                    logging.info(f"🚨 STOP-LOSS triggered at step {i} | Drop: {round(price_change*100, 2)}%")
                    risk_triggered = "STOP-LOSS"
                    entry_price = None  # Reset after exit

                elif price_change > TAKE_PROFIT_PCT:
                    balance = position * price
                    position = 0
                    trade_log.append("TAKE-PROFIT")
                    reward += 0.2
                    logging.info(f"💰 TAKE-PROFIT triggered at step {i} | Gain: {round(price_change*100, 2)}%")
                    risk_triggered = "TAKE-PROFIT"
                    entry_price = None


            current_step = i

            # Whipsaw penalty setup
            trade_occurred = False
            whipsaw_penalty = 0.0

            # === Confidence Buckets Logging ===
            if 'confidence_buckets' not in locals():
                confidence_buckets = {'Low': 0, 'Mid': 0, 'High': 0}

            abs_action_val = abs(action_val)
            if abs_action_val < 0.2:
                confidence_buckets['Low'] += 1
            elif abs_action_val < 0.5:
                confidence_buckets['Mid'] += 1
            else:
                confidence_buckets['High'] += 1

            # === Trade Execution Logic ===
            if abs_action_val > 0.02:
                recent_returns = df_window['Close'].pct_change(5).fillna(0).iloc[i]
                cooldown_ok = (i - last_trade_step) > TRADE_COOLDOWN_STEPS

                if action_val > 0.01 and position == 0 and cooldown_ok:
                    position = balance / price
                    balance = 0
                    entry_price = price
                    trade_log.append("BUY")
                    trade_occurred = True
                    logging.debug(f"✅ BUY executed at step {i} | Price: {price:.2f} | Entry: {entry_price:.2f}")

                    # Log trade return immediately after BUY (if applicable)
                    if entry_price:
                        trade_return = (price - entry_price) / entry_price
                        trade_returns.append(trade_return)
                        total_reward += reward.item() if hasattr(reward, "item") else reward

                elif action_val < -0.01 and position > 0 and cooldown_ok:
                    balance = position * price
                    position = 0
                    trade_log.append("SELL")
                    trade_occurred = True
                    logging.debug(f"✅ SELL executed at step {i} | Price: {price:.2f}")

                    # Log trade return after SELL
                    if entry_price:
                        trade_return = (price - entry_price) / entry_price
                        trade_returns.append(trade_return)
                        total_reward += reward.item() if hasattr(reward, "item") else reward

                else:
                    logging.debug(
                        f"🚫 Trade skipped at step {i} | Action: {action_val:.4f}, Position: {position}, Cooldown OK: {cooldown_ok}"
                    )
                    trade_log.append("HOLD")

            else:
                trade_log.append("HOLD")
                if (i - last_trade_step) <= TRADE_COOLDOWN_STEPS:
                    cooldown_skips.append(i)
                    print(f"⏳ Trade skipped due to cooldown at step {i} | Δ={i - last_trade_step}")



            # Apply whipsaw penalty if trade occurred soon after last trade
            if trade_occurred:
                steps_since_last_trade = current_step - last_trade_step
                if steps_since_last_trade < 30:  # Example: penalize trades within 10 steps
                    whipsaw_penalty = -0.4  # Customize penalty size
                last_trade_step = current_step

            portfolio_value = balance if balance > 0 else position * price
            reward += whipsaw_penalty
            reward = np.clip(reward, -1.0, 1.0)
            if risk_triggered:
                 risk_flags.append((i, risk_triggered))
            if risk_flags:
                logging.info(f"📊 Risk events triggered for {ticker} | Total: {len(risk_flags)} | Details: {risk_flags[:5]}")

            if whipsaw_penalty < 0:
                 logging.info(f"⚠️ Whipsaw penalty applied at step {i} | Δ={steps_since_last_trade}")

            portfolio.append(portfolio_value)

        if not portfolio:
            logging.warning(f"⚠️ Empty portfolio for {ticker} | Window {start}-{end}")
            continue

        if 'confidence_buckets' in locals():
            print(f"🧠 Confidence Buckets for {ticker} | Window {start}-{end}: {confidence_buckets}")

        plt.hist(action_vals, bins=20)
        plt.title(f"Action Distribution for {ticker} | Window {start}-{end}")
        plt.xlabel("Action Value")
        plt.ylabel("Frequency")
        plt.grid(True, linestyle="--", alpha=0.6)
        plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_DIR, f"{ticker}_action_distribution_window{w_idx+1}.png"))
        plt.close()

        # Metrics
        final_value = portfolio[-1]
        hold_value = (100000 / df_window['Close'].iloc[0]) * df_window['Close'].iloc[-1]
        returns = pd.Series(portfolio).pct_change().fillna(0)
        sharpe = (returns.mean() / (returns.std() + 1e-6)) * np.sqrt(252)
        drawdown = ((pd.Series(portfolio).cummax() - pd.Series(portfolio)) / pd.Series(portfolio).cummax()).max() * 100

        # ✅ Hit Ratio Calculation
        successful_trades = sum([1 for t in trade_log if t in ['TAKE-PROFIT']])
        total_trades = sum([1 for t in trade_log if t in ['SELL', 'STOP-LOSS', 'TAKE-PROFIT']])
        hit_ratio = successful_trades / (total_trades + 1e-6)
        print(f"🎯 Hit Ratio: {hit_ratio:.2%}")

        # === Diagnostics
        print(f"\n🧾 {ticker} | Trades Summary (Window {start}-{end}):")
        print(pd.Series(trade_log).value_counts())
        print(f"💼 Final Portfolio Value: {final_value:.2f}")
        print(f"📈 Buy & Hold Value: {hold_value:.2f}")
        print(f"📊 Sharpe: {sharpe:.3f} | Drawdown: {drawdown:.2f}%")
        if risk_flags:
            print(f"⚠️ Risk Events Triggered: {len(risk_flags)} | Examples: {risk_flags[:3]}")
        if cooldown_skips:
            print(f"⏳ Cooldown Skipped Steps: {len(cooldown_skips)} | Sample: {cooldown_skips[:3]}")
        # === Post-trading metrics (after walkforward loop per window) ===
        if trade_returns:
            print(f"📊 Total Reward Accumulated: {total_reward:.2f}")
            print(f"📈 Avg Trade Return: {np.mean(trade_returns):.2%} | Num Trades: {len(trade_returns)}")
        else:
            print("⚠️ No trades recorded for this window.")
        results.append({
            "Ticker": ticker,
            "Window": f"{start}-{end}",
            "PPO_Portfolio": round(final_value, 2),
            "BuyHold": round(hold_value, 2),
            "Sharpe": round(sharpe, 3),
            "Drawdown_%": round(drawdown, 2),
            "Winner": "PPO" if final_value > hold_value else "Buy & Hold"
        })

        # === Save Plot
        plot_performance(ticker, portfolio, hold_value)
        logging.info(f"⏱️ {ticker} | Window {w_idx+1} runtime: {round(time.time() - window_start_time, 2)} sec")

        # === Save VecNormalize stats
        vecnorm_path = os.path.join(FINAL_MODEL_DIR, f"vecnorm_{ticker}_window{w_idx+1}.pkl")
        if isinstance(env, VecNormalize):
            env.save(vecnorm_path)

        del env, model, df_window, portfolio, trade_log
        gc.collect()
        torch.cuda.empty_cache()

    return results

# === Ticker Processor ===
def process_ticker(ticker):
    print(f"\n🔁 Processing ticker: {ticker}")
    logging.info(f"🔄 Starting PPO Walkforward for {ticker}")
    df_ticker = df[df['Symbol'] == ticker].copy()
    if df_ticker.empty:
        print(f"⚠️ No data found for {ticker}")
        return []
    return walkforward_ppo(df_ticker, ticker)

# === Parallel Runner ===
def run_parallel_tickers(tickers, out_path=os.path.join(RESULTS_DIR, "summary.csv")):
    completed = set()
    if os.path.exists(out_path) and os.path.getsize(out_path) > 0:
        try:
            completed_df = pd.read_csv(out_path)
            completed = set(completed_df['Ticker'].unique())
        except Exception as e:
            print(f" Warning reading {out_path}: {e}")

    tickers_to_run = [t for t in tickers if t not in completed]
    print(f"Tickers to run: {tickers_to_run}")
    results = []

    with ThreadPoolExecutor(max_workers=min(len(tickers_to_run), os.cpu_count())) as executor:
        for result in executor.map(process_ticker, tickers_to_run):
            if result:
                results.extend(result)
                pd.DataFrame(results).to_csv(out_path, index=False)

    print("\n Final summary saved.")
    logging.info(f"✅ Completed PPO Walkforward for {ticker} | Total Windows: {len(windows)}")
    return results

# === Final Execution Block ===
CONFIG = {
    'symbols': df['Symbol'].unique().tolist()
}

if test_mode:
    print("\n🧪 Test mode enabled: Running AAPL, NVDA, MSFT")
    test_stocks = ['AAPL', 'NVDA', 'MSFT']
    all_results = []
    for stock in test_stocks:
        df_stock = df[df['Symbol'] == stock].copy()
        results = walkforward_ppo(df_stock, stock)
        all_results.extend(results)
    pd.DataFrame(all_results).to_csv(os.path.join(RESULTS_DIR, "summary_test_mode.csv"), index=False)
else:
    summary_df = run_parallel_tickers(CONFIG['symbols'])
    if summary_df:
        df_summary = pd.DataFrame(summary_df)
        if not df_summary.empty and "PPO_Portfolio" in df_summary.columns:
            print(df_summary.sort_values("PPO_Portfolio", ascending=False).head())
        else:
            print(" No valid PPO portfolio results in the summary.")
    else:
        print(" No PPO walkforward results were generated. Check data availability.")



🧪 Test mode enabled: Running AAPL, NVDA, MSFT


KeyboardInterrupt: 